<a href="https://colab.research.google.com/github/yunjeong-chang/smarcle-winter-study/blob/main/K%EA%B2%B9%20%EA%B5%90%EC%B0%A8%20%EA%B2%80%EC%A6%9D%20%EB%B0%9C%ED%91%9C%20%EC%8B%9C%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<13장 과적합 피하기 中 초음파 광물 예측하기 : K겹 교차 검증>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.compat.v1.set_random_seed(3)

# 데이터 적용
dataset = pd.read_csv('sonar.csv', header=None)

X = dataset.iloc[:,:-1]
Y_obj = dataset.iloc[:,-1]

# 문자열 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

"여기까지는 앞의 실습과 내용 동일"


---
" K겹 교차 검증에 필요한 코드 시작! "

*   K = 10 으로 설정
*   StratifiedKFold() 함수 : 데이터를 원하는 숫자만큼 쪼개 각각 학습셋과 테스트셋으로 사용하게 만드는 함수
> 1. n_splits : 몇 개로 분할 할 지
  2. shuffle : fold를 나누기 전 무작위로 섞어 줌
  3. random_state : 난수 발생을 위한 seed의 인자값

In [ ]:
# 10개의 파일로 쪼갬
n_fold = 10

#계층화 K겹 교차 검증
skf = StratifiedKFold(n_splits = n_fold, shuffle= True, random_state= seed)

# 빈 accuracy 배열 (각 폴드 별 정확도를 이 배열에 모아 한 번에 보여줄 수 있게)
accuracy = []

# sum_accuracy 생성 (후에 정확도의 평균 구하기 위해)
sum_accuracy = 0

*  for 구문 : 모델을 만들고 실행하는 부분을 for문으로 묶어 n_fold = 10 만큼 반복되게 함
*  model.fit(x, y, epochs, batch_size) 함수 : 모델 학습 시키는 함수
> *   x : 입력 데이터
  *   y : 값
  * epochs : 학습 반복 횟수
  * batch_size : 몇 문항을 풀고 해답을 맞춰볼지 -> 가중치 갱신

*   model.evalate(x, y) 함수 : 테스트셋으로 모델 평가하는 함수



In [ ]:
for train, test in skf.split(X, Y):
  #print(train)
  #print(test)

In [ ]:
# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y): #주어진 데이터를 학습셋과 테스트셋으로 분리

    model = Sequential()                                           #여기서 부터
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy']) #여기까지는 앞의 실습 코드와 동일

   # 학습셋을 이용하여 학습시키기
    model.fit(X.iloc[train], Y[train], epochs= 100, batch_size= 5)       
    
   # 테스트셋을 이용하여 검증
    k_accuracy = "%.4f" % (model.evaluate(X.iloc[test], Y[test])[1])      

   # 검증한 결과 accuracy 배열에 추가
    accuracy.append(k_accuracy)

   # 검증한 결과 float으로 변환하여 합하기
    sum_accuracy += float(k_accuracy)

In [ ]:
# 결과 출력
print("%.f fold accuracy: " %n_fold, accuracy)

In [ ]:
# 결과에 따른 평균 정확도 구하기
print("각 정확도의 합:", sum_accuracy)
print("\n최종 평균 정확도: %.4f" %(sum_accuracy/len(accuracy)))
print("\n최종 평균 정확도: %.4f" %(np.mean(np.array(accuracy).astype(float))))